In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv(r"C:\Users\hp\Desktop\Tech Mango\Datasets\course_descriptions.csv",encoding="ANSI")

In [3]:
def create_course_desc(row):
    return f'''The course name is {row["course_name"]}, The slug is {row["course_slug"]}, The technology is {row["course_technology"]} and The course topic is {row["course_topic"]}'''

In [4]:
pd.set_option('display.max_rows', 110)
dataset['New_course_description'] = dataset.apply(create_course_desc, axis=1)
print(dataset["New_course_description"])

0      The course name is Introduction to Tableau, Th...
1      The course name is The Complete Data Visualiza...
2      The course name is Introduction to R Programmi...
3      The course name is Data Preprocessing with Num...
4      The course name is Introduction to Data and Da...
5      The course name is Data Cleaning and Preproces...
6      The course name is Introduction to Business An...
7      The course name is Data Analysis with Excel Pi...
8      The course name is SQL, The slug is sql, The t...
9      The course name is Credit Risk Modeling in Pyt...
10     The course name is Python Programmer Bootcamp,...
11     The course name is SQL + Tableau + Python, The...
12     The course name is Introduction to Jupyter, Th...
13     The course name is Statistics, The slug is sta...
14     The course name is Mathematics, The slug is ma...
15     The course name is Introduction to Excel, The ...
16     The course name is Probability, The slug is pr...
17     The course name is Start

In [6]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os
from dotenv import load_dotenv,find_dotenv
from sentence_transformers import SentenceTransformer

C:\Users\hp\.conda\envs\langchain_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
%load_ext dotenv
%dotenv

In [8]:
load_dotenv(find_dotenv(), override = True)

True

In [9]:
pc = Pinecone(api_key = os.environ.get("PINECONE_API"),environment = os.environ.get("PINECONE_ENV"))

In [10]:
pc.list_indexes()

[
    {
        "name": "my-pinecone",
        "metric": "cosine",
        "host": "my-pinecone-y3e23wn.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 384,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "my-pinecone-1",
        "metric": "euclidean",
        "host": "my-pinecone-1-y3e23wn.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": null
    },
    {


In [11]:
pinecone_name= "my-pinecone"
dimension = 384
metric = "cosine"

In [12]:
if pinecone_name in [index.name for index in pc.list_indexes()]:
    pc.delete_index(pinecone_name)
    print(f"{pinecone_name} Succesfully Deleted.")
else:
    print(f"{pinecone_name} Not in index.")

my-pinecone Succesfully Deleted.


In [13]:
pc.create_index(
    name = pinecone_name,
    dimension = dimension,
    metric = metric,
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1")
    )

{
    "name": "my-pinecone",
    "metric": "cosine",
    "host": "my-pinecone-y3e23wn.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [14]:
index = pc.Index(pinecone_name)

In [15]:
model = SentenceTransformer("all-MiniLM-l6-v2")

In [20]:
def create_embedding(row):
    combined_text = ' '.join([str(row[field]) for field in ['course_description', 'New_course_description','course_description_short']])
    embedding = model.encode(combined_text, show_progress_bar = False)
    return embedding

In [19]:
dataset.head()

,course_name,course_slug,course_technology,course_description,course_topic,course_description_short,New_course_description
0,Introduction to Tableau,tableau,tableau,Tableau is now one of the most popular busines...,data visualization,Teaching you how to tell compelling stories wi...,"The course name is Introduction to Tableau, Th..."
1,The Complete Data Visualization Course with Py...,data-visualization,python,The Data Visualization course is designed for ...,data visualization,Teaching you how to master the art of creating...,The course name is The Complete Data Visualiza...
2,Introduction to R Programming,introduction-to-r-programming,r,R is one of the best programming languages spe...,programming,"Providing you with the skills to manipulate, a...",The course name is Introduction to R Programmi...
3,Data Preprocessing with NumPy,data-preprocessing-numpy,python,This course is designed to show you how to wor...,data processing,This course will guide you through one of Pyth...,The course name is Data Preprocessing with Num...
4,Introduction to Data and Data Science,intro-to-data-and-data-science,theory,Working with data is an essential part of main...,machine learning,Introducing you to the field of data science a...,The course name is Introduction to Data and Da...


In [21]:
dataset["embedding"] = dataset.apply(create_embedding, axis = 1)

In [24]:
vector_to_upsert = [(str(row['course_name']), row['embedding'].tolist()) for _, row in dataset.iterrows()]
index.upsert(vectors = vector_to_upsert)

print("Data upserted to pinecone")

Data upserted to pinecone


In [25]:
query = "clustering"
query_embedding = model.encode(query, show_progress_bar = False).tolist()

In [26]:
query_result = index.query(
    vector = [query_embedding],
    top_k = 12,
    include_values= True
)

In [27]:
query_result

{'matches': [{'id': 'Machine Learning in Excel',
              'score': 0.354685217,
              'values': [-0.0183002315,
                         -0.0279485844,
                         -0.0253203623,
                         -0.0126938745,
                         -0.024036685,
                         -0.0219841097,
                         -0.051123675,
                         -0.0535800196,
                         0.00997649692,
                         0.0282286815,
                         -0.040832445,
                         -0.036268644,
                         0.0683277175,
                         -0.0348471217,
                         -0.00728514744,
                         0.0366662927,
                         -0.0033102422,
                         -0.00411817571,
                         -4.75427878e-05,
                         -0.0627969131,
                         0.0846960172,
                         0.0300104823,
                         -0.0528305061,


In [29]:
for match in query_result['matches']:
    print(f"Matched item ID: {match['id']} , score : {match['score']}") 

Matched item ID: Machine Learning in Excel , score : 0.354685217
Matched item ID: Machine Learning with K-Nearest Neighbors , score : 0.313649148
Matched item ID: Machine Learning in Python , score : 0.282944262
Matched item ID: Customer Churn Analysis with SQL and Tableau , score : 0.281879723
Matched item ID: Growth Analysis with SQL, Python, and Tableau   , score : 0.26017341
Matched item ID: Linear Algebra and Feature Selection , score : 0.258531898
Matched item ID: Customer Engagement Analysis with SQL and Tableau , score : 0.234880671
Matched item ID: Fashion Analytics with Tableau , score : 0.233549267
Matched item ID: Machine Learning with Naive Bayes , score : 0.22770685
Matched item ID: Machine Learning with Support Vector Machines , score : 0.225507811
Matched item ID: Data Preprocessing with NumPy , score : 0.219204485
Matched item ID: Data Analysis with Excel Pivot Tables , score : 0.217148945


In [42]:
dataset.head()

,course_name,course_slug,course_technology,course_description,course_topic,course_description_short,New_course_description,embedding
0,Introduction to Tableau,tableau,tableau,Tableau is now one of the most popular busines...,data visualization,Teaching you how to tell compelling stories wi...,"The course name is Introduction to Tableau, Th...","[0.03636873, -0.027419977, -0.081906594, -0.00..."
1,The Complete Data Visualization Course with Py...,data-visualization,python,The Data Visualization course is designed for ...,data visualization,Teaching you how to master the art of creating...,The course name is The Complete Data Visualiza...,"[0.046510328, -0.020395719, -0.018906789, -0.0..."
2,Introduction to R Programming,introduction-to-r-programming,r,R is one of the best programming languages spe...,programming,"Providing you with the skills to manipulate, a...",The course name is Introduction to R Programmi...,"[-0.043807, 0.0059339316, -0.0884467, 0.029377..."
3,Data Preprocessing with NumPy,data-preprocessing-numpy,python,This course is designed to show you how to wor...,data processing,This course will guide you through one of Pyth...,The course name is Data Preprocessing with Num...,"[-0.056979485, -0.0081792865, -0.039837647, -0..."
4,Introduction to Data and Data Science,intro-to-data-and-data-science,theory,Working with data is an essential part of main...,machine learning,Introducing you to the field of data science a...,The course name is Introduction to Data and Da...,"[-0.021475803, 0.020705577, -0.0025892728, 0.0..."
